# Simple Chatbot from Scratch in Python using NLTK and search in web




## NLP
NLP is a way for computers to analyze, understand, and derive meaning from human language in a smart and useful way. By utilizing NLP, developers can organize and structure knowledge to perform tasks such as automatic summarization, translation, named entity recognition, relationship extraction, sentiment analysis, speech recognition, and topic segmentation.

## Import necessary libraries

In [1]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import requests
warnings.filterwarnings('ignore')

## Downloading and installing NLTK




### Installing NLTK Packages




In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

## Reading in the corpus

For our example,we will be using the Wikipedia page for chatbots as our corpus. Copy the contents from the page and place it in a text file named ‘chatbot.txt’. However, you can use any corpus of your choice.

In [3]:
directions='./Chatbot'

In [4]:
f=open(directions+'chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

## Tokenisation

In [5]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

## Preprocessing

We shall now define a function called LemTokens which will take as input the tokens and return normalized tokens.

In [6]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## Keyword matching

Next, we shall define a function for a greeting by the bot i.e if a user’s input is a greeting, the bot shall return a greeting response.ELIZA uses a simple keyword matching for greetings. We will utilize the same concept here.

In [7]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

## get response
To generate a response from our bot for input questions, the concept of document similarity will be used. We define a function response which searches the user’s utterance for one or more known keywords and returns one of several possible responses. If it doesn’t find the input matching any of the keywords, it returns a response:” I am sorry! I don’t understand you”

In [8]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        #robo_response=robo_response+"I am sorry! I don't understand you"
        robo_response=cherche(user_response)
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response



## search  in internet 

In [9]:
def Gsearch(cherche):


  count = 0
  url=''
  try :
      from googlesearch import search
  except ImportError:
      print("No Module named 'google' Found")
  for i in search(query=cherche,tld='co.in',lang='en',num=10,stop=1,pause=2):
      count += 1
            #print (count)
            #print(i + '\n')
      url+=i
            #print(url)
  return url


In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
# link for extract html data
def getdata(url):
	r = requests.get(url)
	return r.text
  

In [11]:
#la premier paragraphe

def get2_info(htmldata):

  soup = BeautifulSoup(htmldata, 'html.parser')
  data = ''
  for data in soup.find_all("p"):
	  return data.get_text()


In [12]:
def cherche(cherche):

  test=Gsearch(cherche)
  htmldata = getdata(test)
  info=get2_info(htmldata)
  return info
cherche('what is data?')

'Data are units of information, often numeric, that are collected through observation.[1] In a more technical sense, data are a set of values of qualitative or quantitative variables about one or more persons or objects,[1] while a datum (singular of data) is a single value of a single variable.[2]\n'

In [21]:

from gtts import gTTS 
import os
from playsound import playsound
#text = 'Global warming is the long-term rise in \n the average temperature of the Earth’s climate system'
#language = 'en'

In [22]:
def text2speech(text,language ):
    speech = gTTS(text = text, lang = language, slow = False)
    if os.path.exists(directions+"text.mp3"):
        remov=os.remove(directions+"text.mp3")
        if remov==None:
            speech.save(directions+'text.mp3')
    else:
        speech.save(directions+'text.mp3')
    #os.system('start text.mp3')
    playsound(directions+"text.mp3")


Finally, we will feed the lines that we want our bot to say while starting and ending a conversation depending upon user’s input.

In [27]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    text=''
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' or user_response=='OK' ):
            flag=False
            print("ROBO: You are welcome..")
            text2speech('You are welcome','en' )
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
                text=greeting(user_response)
                text2speech(text,'en' )
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
what is chatbot
ROBO: design
the chatbot design is the process that defines the interaction between the user and the chatbot.the chatbot designer will define the chatbot personality, the questions that will be asked to the users, and the overall interaction.it can be viewed as a subset of the conversational design.
bye
ROBO: Bye! take care..
